# 動的探索的データ分析ダッシュボード（特徴量付与版）
================================================
06_final_enriched_20250701_20250930.csv を分析

- 132列の特徴量を活用
- 商品別売上予測の準備
- カレンダー・気象・季節変動の影響分析

## 1. 環境設定とライブラリインポート

In [ ]:
# 日本語フォント設定（共通モジュール）
import warnings
warnings.filterwarnings('ignore')

# よく使うライブラリを先に読み込む
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ウィジェットの有無を通知・フラグ化
try:
    import ipywidgets as widgets
    from IPython.display import display, HTML, clear_output
    WIDGETS_AVAILABLE = True
    print('✅ ipywidgets利用可能')
except Exception:
    WIDGETS_AVAILABLE = False
    print('⚠️ ipywidgets未インストール - 一部機能制限')

import font_setup
JP_FP = font_setup.setup_fonts(show_test=False)


## 2. データ読み込みと前処理

In [ ]:
# データパスの設定
file_path = 'output/06_final_enriched_20250701_20250930.csv'

# データ読み込み
print("📊 データ読み込み中...")
df = pd.read_csv(file_path, encoding='utf-8-sig')
print(f"✅ データ読み込み成功: {len(df):,}行 × {len(df.columns)}列")

# 日付型に変換
df['日付'] = pd.to_datetime(df['日付'])

# 基本情報の表示
print(f"\n📊 データ概要:")
print(f"   期間: {df['日付'].min()} ～ {df['日付'].max()}")
print(f"   店舗数: {df['店舗'].nunique()}")
print(f"   商品数: {df['商品名'].nunique()}")
print(f"   カテゴリ数（大分類）: {df['フェイスくくり大分類'].nunique()}")
print(f"   カテゴリ数（中分類）: {df['フェイスくくり中分類'].nunique()}")
print(f"   カテゴリ数（小分類）: {df['フェイスくくり小分類'].nunique()}")

# 欠損値チェック
print(f"\n欠損値の状況:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'欠損数': missing, '欠損率(%)': missing_pct})
missing_df = missing_df[missing_df['欠損数'] > 0].sort_values('欠損数', ascending=False)
if len(missing_df) > 0:
    print(missing_df.head(10))
else:
    print("   欠損値なし")

In [ ]:
# 列の分類
print("\n📊 列の分類:")

# 基本データ列
basic_cols = ['店舗', 'フェイスくくり大分類', 'フェイスくくり中分類', 'フェイスくくり小分類', '商品名', '日付', '売上数量', '売上金額']

# カレンダー特徴列
calendar_cols = [col for col in df.columns if any(keyword in col for keyword in ['年', '月', '日', '曜日', '週', '休日', '連休', 'GW', '盆', '年末', '給料', '学校', '季節', '四半期'])]

# 気象特徴列
weather_cols = [col for col in df.columns if any(keyword in col for keyword in ['気温', '天気', '降水', '気温差', '猛暑', '真夏', '寒', '雨'])]

# 季節・客数指数列
index_cols = [col for col in df.columns if '指数' in col]

print(f"   基本データ: {len(basic_cols)}列")
print(f"   カレンダー特徴: {len(calendar_cols)}列")
print(f"   気象特徴: {len(weather_cols)}列")
print(f"   季節・客数指数: {len(index_cols)}列")

## 3. 基本統計分析

In [ ]:
# 店舗別基本統計
print("📊 店舗別統計:")

store_stats = df.groupby('店舗').agg({
    '売上金額': ['sum', 'mean', 'std'],
    '売上数量': ['sum', 'mean'],
    '商品名': 'nunique',
    '日付': 'nunique'
}).round(2)

store_stats.columns = ['_'.join(col).strip() for col in store_stats.columns.values]
store_stats = store_stats.reset_index()
store_stats['日販'] = store_stats['売上金額_sum'] / store_stats['日付_nunique']

print(store_stats[['店舗', '日販', '売上金額_sum', '商品名_nunique']])

In [ ]:
# 商品別基本統計（上位商品）
print("\n📊 商品別統計（売上上位20商品）:")

product_stats = df.groupby('商品名').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    'フェイスくくり大分類': 'first',
    'フェイスくくり中分類': 'first',
    '店舗': 'nunique'
})

product_stats = product_stats.sort_values('売上金額', ascending=False)
product_stats['平均単価'] = product_stats['売上金額'] / product_stats['売上数量']

print(product_stats.head(20)[['売上金額', '売上数量', '平均単価', 'フェイスくくり大分類']])

## 4. カレンダー特徴の影響分析

In [ ]:
# 曜日別売上パターン
print("📊 曜日別売上パターン分析:")

# 曜日マッピング
weekday_map = {0: '月', 1: '火', 2: '水', 3: '木', 4: '金', 5: '土', 6: '日'}
df['曜日名'] = df['曜日'].map(weekday_map)

# 曜日別集計
weekday_sales = df.groupby('曜日名')['売上金額'].agg(['sum', 'mean', 'count'])
weekday_order = ['月', '火', '水', '木', '金', '土', '日']
weekday_sales = weekday_sales.reindex(weekday_order)

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 平均売上
bars = axes[0].bar(weekday_order, weekday_sales['mean'], color=COLORS[0])
peak_day = weekday_sales['mean'].idxmax()
low_day = weekday_sales['mean'].idxmin()

for i, day in enumerate(weekday_order):
    if day == peak_day:
        bars[i].set_color('#2ECC71')
    elif day == low_day:
        bars[i].set_color('#E74C3C')

axes[0].set_title('曜日別平均売上（ピーク:緑、最低:赤）', fontsize=14, fontproperties=JP_FP)
axes[0].set_ylabel('平均売上金額', fontproperties=JP_FP)

# 店舗別曜日パターン
for store in df['店舗'].unique():
    store_weekday = df[df['店舗'] == store].groupby('曜日名')['売上金額'].mean()
    store_weekday = store_weekday.reindex(weekday_order)
    axes[1].plot(weekday_order, store_weekday, marker='o', label=store, linewidth=2)

axes[1].set_title('店舗別曜日売上パターン', fontsize=14, fontproperties=JP_FP)
axes[1].set_ylabel('平均売上金額', fontproperties=JP_FP)
axes[1].legend(prop=JP_FP)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nピーク曜日: {peak_day}")
print(f"最低曜日: {low_day}")
print(f"ピーク/最低比: {weekday_sales.loc[peak_day, 'mean'] / weekday_sales.loc[low_day, 'mean']:.2f}倍")

In [ ]:
# 休日効果の分析
print("\n📊 休日効果分析:")

# 休日フラグ別集計
holiday_effect = df.groupby('休日フラグ')['売上金額'].agg(['mean', 'count'])
holiday_effect.index = ['平日', '休日']

print(holiday_effect)
print(f"\n休日の売上は平日の{holiday_effect.loc['休日', 'mean'] / holiday_effect.loc['平日', 'mean']:.2f}倍")

## 5. 気象特徴の影響分析

In [ ]:
# 気温と売上の関係
print("📊 気温と売上の相関分析:")

# 日次集計
daily_data = df.groupby('日付').agg({
    '売上金額': 'sum',
    '最高気温': 'first',
    '最低気温': 'first',
    '平均気温': 'first',
    '降水量': 'first',
    '天気': 'first'
}).reset_index()

# 相関係数
temp_corr = daily_data['平均気温'].corr(daily_data['売上金額'])

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 気温と売上の散布図
scatter = axes[0].scatter(daily_data['平均気温'], daily_data['売上金額'] / 10000,
                         c=daily_data.index, cmap='coolwarm', s=50, alpha=0.7)
axes[0].set_title(f'平均気温と売上の関係（相関: {temp_corr:.3f}）', fontsize=14, fontproperties=JP_FP)
axes[0].set_xlabel('平均気温（℃）', fontproperties=JP_FP)
axes[0].set_ylabel('売上金額（万円）', fontproperties=JP_FP)

# トレンドライン
z = np.polyfit(daily_data['平均気温'], daily_data['売上金額'], 1)
p = np.poly1d(z)
axes[0].plot(daily_data['平均気温'].sort_values(), 
             p(daily_data['平均気温'].sort_values()) / 10000,
             "r--", linewidth=2, alpha=0.8)

# 天気別売上
weather_sales = daily_data.groupby('天気')['売上金額'].mean().sort_values(ascending=False)
bars = axes[1].bar(weather_sales.index, weather_sales.values / 10000, color=COLORS[:len(weather_sales)])
axes[1].set_title('天気別平均売上', fontsize=14, fontproperties=JP_FP)
axes[1].set_ylabel('平均売上金額（万円）', fontproperties=JP_FP)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\n気温と売上の相関係数: {temp_corr:.3f}")
print(f"最も売上が高い天気: {weather_sales.index[0]}")

## 6. 商品カテゴリ分析

In [ ]:
# カテゴリ別パフォーマンス
print("📊 カテゴリ別パフォーマンス:")

category_performance = df.groupby('フェイスくくり大分類').agg({
    '売上金額': 'sum',
    '売上数量': 'sum',
    '商品名': 'nunique'
})

category_performance = category_performance.sort_values('売上金額', ascending=False)
category_performance['平均単価'] = category_performance['売上金額'] / category_performance['売上数量']

print(category_performance.head(10))

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 売上構成
top10 = category_performance.head(10)
axes[0].pie(top10['売上金額'], labels=top10.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('カテゴリ別売上構成（上位10）', fontsize=14, fontproperties=JP_FP)

# 売上ランキング
axes[1].barh(range(len(top10)), top10['売上金額'] / 10000, color=COLORS[:len(top10)])
axes[1].set_yticks(range(len(top10)))
axes[1].set_yticklabels(top10.index)
axes[1].set_xlabel('売上金額（万円）', fontproperties=JP_FP)
axes[1].set_title('カテゴリ別売上ランキング', fontsize=14, fontproperties=JP_FP)
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## 7. 時系列トレンド分析

In [ ]:
# 日次売上トレンド
print("📊 時系列トレンド分析:")

# トレンド分析
x = np.arange(len(daily_data))
y = daily_data['売上金額'].values
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# 可視化
plt.figure(figsize=(14, 6))
plt.plot(daily_data['日付'], daily_data['売上金額'] / 10000, alpha=0.7, label='日次売上')

# トレンドライン
trend_line = slope * x + intercept
plt.plot(daily_data['日付'], trend_line / 10000, 'r--', linewidth=2, 
         label=f'トレンド（傾き: {slope:.0f}円/日）')

# 移動平均
ma7 = daily_data['売上金額'].rolling(window=7, center=True).mean()
plt.plot(daily_data['日付'], ma7 / 10000, 'g-', linewidth=2, label='7日移動平均')

plt.title('売上の時系列トレンド', fontsize=14, fontproperties=JP_FP)
plt.xlabel('日付', fontproperties=JP_FP)
plt.ylabel('売上金額（万円）', fontproperties=JP_FP)
plt.legend(prop=JP_FP)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

trend_direction = 'increasing' if slope > 0 else 'decreasing'
print(f"\nトレンド方向: {trend_direction}")
print(f"日次変化: {slope:.0f}円/日")
print(f"統計的有意性: {p_value < 0.05}")

## 8. 特徴量の重要度分析

In [ ]:
# 特徴量と売上の相関分析
print("📊 特徴量と売上の相関分析:")

# 数値列のみを抽出
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# 売上金額との相関を計算
correlations = df[numeric_cols].corr()['売上金額'].sort_values(ascending=False)
correlations = correlations[correlations.index != '売上金額']  # 自己相関を除外

# 上位・下位10個を表示
print("\n売上との正の相関（上位10）:")
print(correlations.head(10))

print("\n売上との負の相関（下位10）:")
print(correlations.tail(10))

# 可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 正の相関
top_positive = correlations.head(15)
axes[0].barh(range(len(top_positive)), top_positive.values, color=COLORS[0])
axes[0].set_yticks(range(len(top_positive)))
axes[0].set_yticklabels(top_positive.index, fontsize=8)
axes[0].set_xlabel('相関係数', fontproperties=JP_FP)
axes[0].set_title('売上との正の相関（上位15特徴）', fontsize=14, fontproperties=JP_FP)
axes[0].invert_yaxis()

# 負の相関
top_negative = correlations.tail(15)
axes[1].barh(range(len(top_negative)), top_negative.values, color=COLORS[1])
axes[1].set_yticks(range(len(top_negative)))
axes[1].set_yticklabels(top_negative.index, fontsize=8)
axes[1].set_xlabel('相関係数', fontproperties=JP_FP)
axes[1].set_title('売上との負の相関（下位15特徴）', fontsize=14, fontproperties=JP_FP)
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## 9. 統合ダッシュボード

In [ ]:
# 統合ダッシュボード作成
print("📊 統合ダッシュボード生成中...")

fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. 店舗別売上
ax1 = fig.add_subplot(gs[0, 0])
ax1.bar(store_stats['店舗'], store_stats['日販'] / 10000, color=COLORS[0])
ax1.set_ylabel('日販（万円）', fontproperties=JP_FP)
ax1.set_title('店舗別日販', fontsize=12, fontproperties=JP_FP)
ax1.tick_params(axis='x', rotation=45)

# 2. 曜日別売上
ax2 = fig.add_subplot(gs[0, 1])
ax2.bar(weekday_order, weekday_sales['mean'], color=COLORS[1])
ax2.set_ylabel('平均売上金額', fontproperties=JP_FP)
ax2.set_title('曜日別平均売上', fontsize=12, fontproperties=JP_FP)

# 3. カテゴリ別売上構成
ax3 = fig.add_subplot(gs[0, 2])
top5_cat = category_performance.head(5)
ax3.pie(top5_cat['売上金額'], labels=top5_cat.index, autopct='%1.1f%%', startangle=90)
ax3.set_title('カテゴリ別売上（上位5）', fontsize=12, fontproperties=JP_FP)

# 4. 売上トレンド
ax4 = fig.add_subplot(gs[1, :])
ax4.plot(daily_data['日付'], daily_data['売上金額'] / 10000, alpha=0.7)
ax4.plot(daily_data['日付'], ma7 / 10000, 'r-', linewidth=2, label='7日MA')
ax4.set_ylabel('売上金額（万円）', fontproperties=JP_FP)
ax4.set_title('日次売上トレンド', fontsize=12, fontproperties=JP_FP)
ax4.legend(prop=JP_FP)
ax4.grid(True, alpha=0.3)

# 5. 気温と売上の関係
ax5 = fig.add_subplot(gs[2, 0])
ax5.scatter(daily_data['平均気温'], daily_data['売上金額'] / 10000, alpha=0.6)
ax5.set_xlabel('平均気温（℃）', fontproperties=JP_FP)
ax5.set_ylabel('売上金額（万円）', fontproperties=JP_FP)
ax5.set_title(f'気温-売上相関（r={temp_corr:.3f}）', fontsize=12, fontproperties=JP_FP)

# 6. 商品別売上（上位10）
ax6 = fig.add_subplot(gs[2, 1:])
top10_products = product_stats.head(10)
ax6.barh(range(len(top10_products)), top10_products['売上金額'] / 10000, color=COLORS[2])
ax6.set_yticks(range(len(top10_products)))
ax6.set_yticklabels(top10_products.index, fontsize=8)
ax6.set_xlabel('売上金額（万円）', fontproperties=JP_FP)
ax6.set_title('商品別売上ランキング（上位10）', fontsize=12, fontproperties=JP_FP)
ax6.invert_yaxis()

plt.suptitle('動的ビジネスインサイト統合ダッシュボード（特徴量付与版）', fontsize=20, fontproperties=JP_FP)
plt.tight_layout()
plt.show()

## 10. 分析結果サマリー

In [ ]:
# 分析結果の統合
print("\n" + "="*80)
print("📊 分析結果サマリー")
print("="*80)

print(f"\n1. 基本情報:")
print(f"   分析期間: {(df['日付'].max() - df['日付'].min()).days + 1}日間")
print(f"   総売上: {df['売上金額'].sum()/10000:.0f}万円")
print(f"   日販: {df['売上金額'].sum() / daily_data['日付'].nunique() / 10000:.0f}万円/日")
print(f"   特徴量数: {len(df.columns)}列")

print(f"\n2. 店舗パフォーマンス:")
for _, row in store_stats.iterrows():
    print(f"   {row['店舗']}: 日販{row['日販']/10000:.1f}万円（商品数{row['商品名_nunique']:.0f}）")

print(f"\n3. 売上パターン:")
print(f"   ピーク曜日: {peak_day}")
print(f"   最低曜日: {low_day}")
print(f"   休日効果: {holiday_effect.loc['休日', 'mean'] / holiday_effect.loc['平日', 'mean']:.2f}倍")

print(f"\n4. 気象影響:")
print(f"   気温-売上相関: {temp_corr:.3f}")
print(f"   最適天気: {weather_sales.index[0]}")

print(f"\n5. トップカテゴリ:")
for i, (cat, sales) in enumerate(category_performance.head(5)['売上金額'].items(), 1):
    print(f"   {i}. {cat}: {sales/10000:.1f}万円")

print(f"\n6. トップ商品:")
for i, (prod, sales) in enumerate(product_stats.head(5)['売上金額'].items(), 1):
    print(f"   {i}. {prod}: {sales/10000:.1f}万円")

print("\n" + "="*80)
print("✅ 分析完了！次のステップ: 機械学習モデルの構築")
print("="*80)